<a href="https://colab.research.google.com/github/sundaybest3/s24Corpus-final/blob/main/Corpus/TEDdata/TED_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TED data Text pre-processing

+ Last updated (6/4)

# 🍀Todo: Removing timestamp and parenthetical notes in the script

In [ ]:
# Sample input text
text = """
The human voice: It's the instrument we all play. It's the most powerful sound in the world, probably. It's the only one that can start a war or say "I love you." And yet many people have the experience that when they speak, people don't listen to them. And why is that? How can we speak powerfully to make change in the world?

00:24
What I'd like to suggest, there are a number of habits that we need to move away from. I've assembled for your pleasure here seven deadly sins of speaking. I'm not pretending this is an exhaustive list, but these seven, I think, are pretty large habits that we can all fall into.

00:40
First, gossip. Speaking ill of somebody who's not present. Not a nice habit, and we know perfectly well the person gossiping, five minutes later, will be gossiping about us.

00:53
Second, judging. We know people who are like this in conversation, and it's very hard to listen to somebody if you know that you're being judged and found wanting at the same time.

01:03
Third, negativity. You can fall into this. My mother, in the last years of her life, became very negative, and it's hard to listen. I remember one day, I said to her, "It's October 1 today," and she said, "I know, isn't it dreadful?"

01:16
(Laughter)

01:18
It's hard to listen when somebody's that negative.

01:21
(Laughter)
"""

+ This script defines a function clean_text that takes in the raw text as input. It uses the re.sub function to substitute the patterns of timestamps and bracketed text with an empty string, effectively removing them from the text. After defining the function, you can pass any text to it to get the cleaned output.

```
re.sub(r'\d{2}:\d{2}\n', '', text)
```

+ Pattern Explained:
  + \d{2}: This matches exactly two digits. The \d denotes a digit (equivalent to [0-9]), and {2} specifies that exactly two instances of the preceding element (in this case, a digit) should be found.
  + :: This matches the colon character exactly as it appears.
  + \d{2}: This again matches exactly two digits following the colon, corresponding to the minute format in a timestamp.
  + \n: This matches a newline character, ensuring that the pattern matches timestamps that are followed by a line break, which is typical when timestamps are used to label sections of text.
+ Replacement: ' ' (an empty string) — this indicates that every match found by the pattern in the text should be replaced with nothing, effectively removing it.
+ Application: This line of code will search through text for any sequence that matches a time stamp (e.g., "01:21") followed immediately by a newline, and removes those sequences.

In [ ]:
import re

def clean_text(text):
    # Remove timestamps in the format "00:00"
    text = re.sub(r'\d{2}:\d{2}\n', '', text)
    # Remove text within brackets
    text = re.sub(r'\(.*?\)', '', text)
    return text

cleaned_text = clean_text(text)
print(cleaned_text)


# 🍀Todo:

---
# Step by step to get a cleaned text for the text column in our csv file

+ Read csv file as data (using Github link)
+ Read Column 'Text' and remove time stamps and parenthetical notes, and write the cleaned text in a new column named 'Cleantext01'

## Split Speaker info

+ [data sample](https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/sample1.csv)

In [ ]:
#@markdown Read csv, add Speaker information in 'Speaker' column
import pandas as pd
import re

# Load the CSV file
url = "https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/sample1.csv"
# file_path = '/content/sample1.csv'  # Adjust the file path after uploading your file to Colab
data = pd.read_csv(url)

# Function to extract speaker name using regex
def extract_speaker(title):
    # This regex looks for the pattern where the name is in all capitals at the end of the string
    match = re.search(r'\n([A-Z\s]+)$', title)
    if match:
        return match.group(1).strip()  # Return the matched group, stripped of leading/trailing whitespace
    return ''  # Return empty string if no speaker name is found

# Apply the function to separate the 'Speaker' from 'Title'
data['Speaker'] = data['Title'].apply(extract_speaker)

# Remove the speaker name from the 'Title' column
data['Title'] = data['Title'].apply(lambda x: re.sub(r'\n[A-Z\s]+$', '', x))

# Insert the 'Speaker' column right after the 'Title' column
speaker_col = data.pop('Speaker')  # Remove the 'Speaker' column temporarily
data.insert(1, 'Speaker', speaker_col)  # Insert it right after 'Title' column

# Save the updated DataFrame to a new CSV file
output_file_path = '/content/sample1_speaker.csv'
data.to_csv(output_file_path, index=False)

print("Updated DataFrame:")
print(data.head())


## Text Clean-up [1]: removing timestamp and parenthetical notes, and add a new column

```
def clean_text(text):
    # Remove timestamps in the format "00:00"
    text = re.sub(r'\d{2}:\d{2}\n', '', text)
    # Remove text within brackets
    text = re.sub(r'\(.*?\)', '', text)
    return text
```

In [ ]:
# data.head()

import pandas as pd
import re

# Assuming 'data' is your original DataFrame
df = data

def clean_text(text):
    # Remove timestamps in the format "00:00"
    text = re.sub(r'\d{2}:\d{2}\n', '', text)
    # Remove text within brackets
    text = re.sub(r'\(.*?\)', '', text)
    return text

# Apply the clean_text function to each element in the 'Text' column
df['Cleanedtext01'] = df['Text'].apply(clean_text)

# Comparing the first item of 'Text' and 'Cleanedtext01'
original_text = df['Text'].iloc[0][0:1000]  # Access the first item in the 'Text' column
cleaned_text = df['Cleanedtext01'].iloc[0][0:1000]  # Access the first item in the 'Cleanedtext01' column

print("Original Text:")
print(original_text)
print("="*50)
print("\nCleaned Text:")
print(cleaned_text)

### Checking whether the cleaning process is completed as planned

1. find the timestamp expressions
2. find parenthetical notes

In [ ]:
#@markdown 1. Check the first (timestamp) for both 'Text' and 'Cleanedtext01'
import pandas as pd
import re

# Assuming 'df' is your DataFrame
def remove_and_report_timestamps(text):
    # Find all occurrences of the timestamp pattern
    matches = re.findall(r'\d{2}:\d{2}\n', text)
    # Remove the timestamp pattern
    cleaned_text = re.sub(r'\d{2}:\d{2}\n', '', text)
    return cleaned_text, matches

# Apply the function and capture the cleaned text and the matches for 'Text'
cleaned_text_original, timestamp_matches_original = remove_and_report_timestamps(df['Text'][0])

# Print the number of occurrences and list each occurrence for 'Text'
if timestamp_matches_original:
    print(f"Found {len(timestamp_matches_original)} occurrences of the timestamp pattern in original text:")
    for match in timestamp_matches_original:
        print(match.strip())  # .strip() is used to remove any trailing newline for clean display
else:
    print("No timestamp pattern found in the original text.")

# Apply the same function and capture the cleaned text and the matches for 'Cleanedtext01'
cleaned_text_cleaned, timestamp_matches_cleaned = remove_and_report_timestamps(df['Cleanedtext01'][0])

# Print the number of occurrences and list each occurrence for 'Cleanedtext01'
if timestamp_matches_cleaned:
    print(f"Found {len(timestamp_matches_cleaned)} occurrences of the timestamp pattern in cleaned text:")
    for match in timestamp_matches_cleaned:
        print(match.strip())
else:
    print("No timestamp pattern found in the cleaned text.")


In [ ]:
#@markdown 2. Check the second (parenthetical notes) for both 'Text' and 'Cleanedtext01'
import pandas as pd
import re

# Assuming 'df' is your DataFrame
def remove_and_report_timestamps(text):
    # Find all occurrences of the timestamp pattern
    matches = re.findall(r'\(.*?\)', text)
    # Remove the timestamp pattern
    cleaned_text = re.sub(r'\(.*?\)', '', text)
    return cleaned_text, matches

# Apply the function and capture the cleaned text and the matches for 'Text'
cleaned_text_original, timestamp_matches_original = remove_and_report_timestamps(df['Text'][0])

# Print the number of occurrences and list each occurrence for 'Text'
if timestamp_matches_original:
    print(f"Found {len(timestamp_matches_original)} occurrences of the timestamp pattern in original text:")
    for match in timestamp_matches_original:
        print(match.strip())  # .strip() is used to remove any trailing newline for clean display
else:
    print("No timestamp pattern found in the original text.")

# Apply the same function and capture the cleaned text and the matches for 'Cleanedtext01'
cleaned_text_cleaned, timestamp_matches_cleaned = remove_and_report_timestamps(df['Cleanedtext01'][0])

# Print the number of occurrences and list each occurrence for 'Cleanedtext01'
if timestamp_matches_cleaned:
    print(f"Found {len(timestamp_matches_cleaned)} occurrences of the timestamp pattern in cleaned text:")
    for match in timestamp_matches_cleaned:
        print(match.strip())
else:
    print("No parenthetical pattern found in the cleaned text.")


# Find a word in the all text

1. Combine the text and find a word
2. For each text of the data ('Text'), find the word and add a new column with the number of cases found in the given text

In [ ]:
!pip install nltk

### 1a. Combine the text and find 'very' for example

In [ ]:
import pandas as pd
import requests
from io import StringIO
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download necessary NLTK resources
nltk.download('punkt')  # Tokenizer model

# 1) Read a file from URL and assign the file to 'data' dataframe
url = 'https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/sample1.csv'  # Replace with your actual URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# 2) Display column names
print("Column names:", data.columns)
print("="*50)
# 3) Combine all items in the 'Text' column as a single string
combined_text = ''.join(data['Text'].astype(str))

# 4) Save the combined text as 'scriptall.txt'
with open('/content/scriptall.txt', 'w', encoding='utf-8') as file:
    file.write(combined_text)

# 5) Remove punctuation using NLTK and save it as 'scriptall_nopunct.txt'
tokens = word_tokenize(combined_text)
tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation
text_no_punctuation = ' '.join(tokens)
with open('/content/scriptall_nopunct.txt', 'w', encoding='utf-8') as file:
    file.write(text_no_punctuation)

# 6) Search matching strings 'very' (lower or capital) and display left and right 50 characters for all occurrences
pattern = r'\bvery\b'  # Case-sensitive example; add (?i) for case-insensitive
occurrences = 0
for i, word in enumerate(tokens):
    if word.lower() == 'very':
        start = max(0, i - 10)  # Approximate word count before 'very'
        end = min(len(tokens), i + 10)  # Approximate word count after 'very'
        print(' '.join(tokens[start:end]))
        occurrences += 1

# 7) Print summary with how many occurrences are found in the given text
print("="*50)
print(f"Total occurrences found: {occurrences}")


### 1b. Combine the text and find a word (using user input)

In [ ]:
import pandas as pd
import requests
from io import StringIO
import nltk
from nltk.tokenize import word_tokenize
import string

# Download necessary NLTK resources
nltk.download('punkt')  # Tokenizer model

# 1) Read a file from URL and assign the file to 'data' dataframe
url = 'https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/sample1.csv'
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# 2) Display column names
print("Column names:", data.columns)
print("="*50)

# 3) Combine all items in the 'Text' column as a single string
combined_text = ''.join(data['Text'].astype(str))

# 4) Save the combined text as 'scriptall.txt'
with open('/content/scriptall.txt', 'w', encoding='utf-8') as file:
    file.write(combined_text)

# 5) Remove punctuation using NLTK and save it as 'scriptall_nopunct.txt'
tokens = word_tokenize(combined_text)
tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation
text_no_punctuation = ' '.join(tokens)
with open('/content/scriptall_nopunct.txt', 'w', encoding='utf-8') as file:
    file.write(text_no_punctuation)

# Get user input for the word to find
search_word = input("Enter the word to find: ")

# 6) Search for the input word and display left and right 50 characters (approx. 10 words) for all occurrences
occurrences = 0
for i, word in enumerate(tokens):
    if word.lower() == search_word.lower():
        start = max(0, i - 10)  # Approximate word count before the search word
        end = min(len(tokens), i + 10)  # Approximate word count after the search word
        print(' '.join(tokens[start:end]))
        occurrences += 1

# 7) Print summary with how many occurrences are found in the given text
print("="*50)
print(f"Total occurrences found: {occurrences}")


### 2. Find a word for each text and add the information as a separate column named 'CountVery'

In [ ]:
#@markdown User input for a word to search, user input for the column name to record the number of occurrences
import pandas as pd
import requests
from io import StringIO
import nltk
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download('punkt')  # Tokenizer model

# 1) Read a file from URL and assign the file to 'data' dataframe
url = 'https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/sample1.csv'
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# 2) Display column names
print("Column names:", data.columns)
print("="*50)

# Get user input for the word to find and the new column name
search_word = input("Enter the word to find: ").lower()
new_column_name = input("Enter the new column name for word occurrences: ")

# 3) Define a function to count occurrences of a specified word in a text
def count_word_occurrences(text, word):
    tokens = word_tokenize(text)
    count = sum(1 for token in tokens if token.lower() == word)
    return count

# 4) Apply this function to each item in the 'Text' column and add the result to a new column
data[new_column_name] = data['Text'].apply(lambda text: count_word_occurrences(text, search_word))

# Display the updated DataFrame
print(data[[new_column_name]].head())

# Optionally, save the updated DataFrame to a new CSV file
# data.to_csv('/content/updated_data.csv', index=False)


# **Final data to process**

## [1] Data to read

[data link](https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/TED100.csv)

In [ ]:
import pandas as pd

datalink = "https://raw.githubusercontent.com/MK316/Spring2024/main/Corpus/TEDdata/TED100.csv"
data = pd.read_csv(datalink, encoding="utf-8")
data.head()

## [2] Cleaned data: adding a column 'Cleanedtext01'

+ data = original data
+ df = cleaned data column added

In [ ]:

import pandas as pd
import re

# Assuming 'data' is your original DataFrame
df = data

def clean_text(text):
    # Remove timestamps in the format "00:00"
    text = re.sub(r'\d{2}:\d{2}\n', '', text)
    # Remove text within brackets
    text = re.sub(r'\(.*?\)', '', text)
    return text

# Apply the clean_text function to each element in the 'Text' column
df['Cleanedtext01'] = df['Text'].apply(clean_text)

# Comparing the first item of 'Text' and 'Cleanedtext01'
original_text = df['Text'].iloc[0][0:1000]  # Access the first item in the 'Text' column
cleaned_text = df['Cleanedtext01'].iloc[0][0:1000]  # Access the first item in the 'Cleanedtext01' column

print("Original Text:")
print(original_text)
print("="*50)
print("\nCleaned Text:")
print(cleaned_text)

## [3] Check whether the data cleaning is appropriately processed

In [ ]:
#@markdown 1. Check the first (timestamp) for both 'Text' and 'Cleanedtext01'
import pandas as pd
import re

# Assuming 'df' is your DataFrame
def remove_and_report_timestamps(text):
    # Find all occurrences of the timestamp pattern
    matches = re.findall(r'\d{2}:\d{2}\n', text)
    # Remove the timestamp pattern
    cleaned_text = re.sub(r'\d{2}:\d{2}\n', '', text)
    return cleaned_text, matches

# Apply the function and capture the cleaned text and the matches for 'Text'
tn = input("Type the index of a text to check (1~100): ")
tn = int(tn)
n = tn-1
cleaned_text_original, timestamp_matches_original = remove_and_report_timestamps(df['Text'][n])

# Print the number of occurrences and list each occurrence for 'Text'
if timestamp_matches_original:
    print(f"Found {len(timestamp_matches_original)} occurrences of the timestamp pattern in original text:")
    for match in timestamp_matches_original:
        print(match.strip())  # .strip() is used to remove any trailing newline for clean display
else:
    print("No timestamp pattern found in the original text.")

# Apply the same function and capture the cleaned text and the matches for 'Cleanedtext01'
cleaned_text_cleaned, timestamp_matches_cleaned = remove_and_report_timestamps(df['Cleanedtext01'][n])

# Print the number of occurrences and list each occurrence for 'Cleanedtext01'
if timestamp_matches_cleaned:
    print(f"Found {len(timestamp_matches_cleaned)} occurrences of the timestamp pattern in cleaned text:")
    for match in timestamp_matches_cleaned:
        print(match.strip())
else:
    print("No timestamp pattern found in the cleaned text.")


In [ ]:
#@markdown 2. Check the second (parenthetical notes) for both 'Text' and 'Cleanedtext01'
import pandas as pd
import re

# Assuming 'df' is your DataFrame
def remove_and_report_timestamps(text):
    # Find all occurrences of the timestamp pattern
    matches = re.findall(r'\(.*?\)', text)
    # Remove the timestamp pattern
    cleaned_text = re.sub(r'\(.*?\)', '', text)
    return cleaned_text, matches

# Apply the function and capture the cleaned text and the matches for 'Text'
ts = input("Which text to check (1~100): ")
ts = int(ts)
s = ts-1
cleaned_text_original, timestamp_matches_original = remove_and_report_timestamps(df['Text'][s])

# Print the number of occurrences and list each occurrence for 'Text'
if timestamp_matches_original:
    print(f"Found {len(timestamp_matches_original)} occurrences of the timestamp pattern in original text:")
    for match in timestamp_matches_original:
        print(match.strip())  # .strip() is used to remove any trailing newline for clean display
else:
    print("No timestamp pattern found in the original text.")

# Apply the same function and capture the cleaned text and the matches for 'Cleanedtext01'
cleaned_text_cleaned, timestamp_matches_cleaned = remove_and_report_timestamps(df['Cleanedtext01'][s])

# Print the number of occurrences and list each occurrence for 'Cleanedtext01'
if timestamp_matches_cleaned:
    print(f"Found {len(timestamp_matches_cleaned)} occurrences of the timestamp pattern in cleaned text:")
    for match in timestamp_matches_cleaned:
        print(match.strip())
else:
    print("No parenthetical pattern found in the cleaned text.")


## [4] Text to combine for searching (to check)

In [ ]:
import pandas as pd
import requests
from io import StringIO
import string


# 2) Combine all items in the 'Text' column as a single string and remove punctuation
combined_text = ''.join(df['Cleanedtext01'].astype(str))
combined_text = combined_text.translate(str.maketrans('', '', string.punctuation))

# 3) Get user input for the word to find
search_word = input("Enter the word to find: ")
match_type = input("Type 'c' complete matches only, or 'p' for partial matches: ").lower()

# 4) Function to find occurrences
def find_occurrences(text, word, match_type):
    occurrences = []
    position = 0
    while True:
        if match_type == 'c':
            # Find whole words only by using boundaries
            position = text.lower().find(f' {word.lower()} ', position)
        else:
            position = text.lower().find(word.lower(), position)

        if position == -1:  # No more occurrences found
            break
        # Calculate start and end positions for slicing
        start = max(0, position - 30)
        end = min(len(text), position + len(word) + 30)
        occurrences.append(text[start:end])
        position += len(word)  # Move past this occurrence

    return occurrences

occurrences = find_occurrences(combined_text, search_word, match_type)

# 5) Decide how many occurrences to display
print(f"Total occurrences found: {len(occurrences)}")
print("="*50)
if len(occurrences) > 10:
    choice = input("More than 10 occurrences found. Type 'a' to display all or '10' to display only the first 10: ").lower()
    print("="*50)
    if choice == '10':
        occurrences = occurrences[:10]

# 6) Display occurrences
for occurrence in occurrences:
    print(occurrence)

# 7) Print summary
print("="*50)


# Saving the processed file

In [ ]:
print(df.head())
df.to_csv("Cleanedtext01.csv", encoding = "utf-8", index=False)